In [1]:
!pip install earthengine-api

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import ee
import os
import csv
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm  # For progress bar visualization

ee.Authenticate()
ee.Initialize(project='reuben-datathon-s')

################################ COMBINED ################################

# Define the function to retrieve data for each location
def allCombined(longitude, latitude, start_date, end_date):
    ####### ALBEDO #######
    dataset = ee.ImageCollection('MODIS/061/MCD43A3').select('Albedo_BSA_shortwave')  # Updated dataset version
    point = ee.Geometry.Point([longitude, latitude]).buffer(500)
    region = point.buffer(500)
    filtered_dataset = dataset.filterDate(start_date, end_date).mean()
    albedo_values = filtered_dataset.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region,
        scale=500,
        maxPixels=1e13
    ).get('Albedo_BSA_shortwave')
    albedo_dict = albedo_values.getInfo()

    ####### Mean AAI #######
    aoi = ee.Geometry.Point([longitude, latitude]).buffer(500)
    dataset = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_AER_AI').filterBounds(aoi).filterDate(start_date, end_date)
    clipped_dataset = dataset.map(lambda img: img.clip(aoi))
    aai_band = clipped_dataset.select('absorbing_aerosol_index')
    mean_aai = aai_band.mean()
    mean_value = mean_aai.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=500
    ).get('absorbing_aerosol_index')
    overall_mean_aai = mean_value.getInfo()

    ####### Vegetation Area #######
    dataset = ee.ImageCollection('JRC/GHSL/P2023A/GHS_BUILT_C').filterBounds(aoi)
    built_up_layer = dataset.first().clip(aoi)
    def calculate_area_for_class(class_value):
        class_mask = built_up_layer.eq(class_value)
        pixel_area = ee.Image.pixelArea().updateMask(class_mask)
        area = pixel_area.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=aoi,
            scale=30,
            maxPixels=1e10
        ).get('area')
        return ee.Number(area).divide(1e6)
    area_low_veg = calculate_area_for_class(1)
    area_medium_veg = calculate_area_for_class(2)
    total_veg_area = area_low_veg.add(area_medium_veg)
    total_aoi_area = ee.Number(aoi.area().divide(1e6))
    ratio_veg_to_aoi = total_veg_area.divide(total_aoi_area)
    ratio_veg_to_aoi_info = ratio_veg_to_aoi.getInfo()

    ####### Population Density #######
    population_count = ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Count").first().clip(aoi)
    land_area_dataset = ee.ImageCollection("CIESIN/GPWv411/GPW_Land_Area").first().clip(aoi)
    total_population = population_count.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=927.67,
        maxPixels=1e10
    ).get('population_count')
    total_land_area = land_area_dataset.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=927.67,
        maxPixels=1e10
    ).get('land_area')
    population_density = ee.Number(total_population).divide(total_land_area)
    density_value = population_density.getInfo()

    ####### Protected Area #######
    dataset = ee.FeatureCollection("WCMC/WDPA/current/polygons")
    pa_def_filtered = dataset.filterBounds(aoi).filterDate(start_date, end_date).filter(ee.Filter.eq('PA_DEF', '1'))
    protected_area_sum = pa_def_filtered.filterBounds(aoi).geometry().area().divide(1e6).getInfo()
    total_aoi_area = aoi.area().divide(1e6).getInfo()
    fraction_protected = protected_area_sum / total_aoi_area if total_aoi_area > 0 else 0

    ####### Slope #######
    location = ee.Geometry.Point([longitude, latitude])
    buffered_area = location.buffer(500)
    dem_dataset = ee.ImageCollection('COPERNICUS/DEM/GLO30').mosaic()
    slope = ee.Terrain.slope(dem_dataset)
    mean_slope = slope.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=buffered_area,
        scale=30,
        maxPixels=1e13
    ).get('slope')
    slope_at_point = mean_slope.getInfo()

    ####### Cloud #######
    modis = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED').filterBounds(aoi)
    modis_mean = modis.filterDate(start_date, end_date).select('cs').mean()
    mean_dict = modis_mean.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=1000,
        maxPixels=1e13
    )
    mean_cloudCoverage = mean_dict.getInfo().get('cs')

    ####### Land Surface Temperature #######
    buffered_area = ee.Geometry.Point([longitude, latitude]).buffer(500)
    dataset = ee.ImageCollection('MODIS/061/MOD11A1').filterDate(start_date, end_date).filterBounds(buffered_area).select('LST_Day_1km')  # Updated to 'MODIS/061/MOD11A1'
    mean_lst = dataset.mean()
    mean_value = mean_lst.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=buffered_area,
        scale=1000,
        maxPixels=1e13
    ).get('LST_Day_1km')
    lst = ((mean_value.getInfo())*0.02)-273.115

    ####### Downward Radiation #######
    location = ee.Geometry.Point([longitude, latitude]).buffer(500)
    dataset = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY').filterDate(start_date, end_date).filterBounds(location).select('surface_solar_radiation_downwards')
    mean_radiation = dataset.mean()
    mean_value = mean_radiation.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=location,
        scale=1000
    ).get('surface_solar_radiation_downwards')
    dr = mean_value.getInfo()

    return albedo_dict, overall_mean_aai, ratio_veg_to_aoi_info, density_value, fraction_protected, slope_at_point, mean_cloudCoverage, lst, dr


# Function to process each row (lat/long) in parallel
def process_row(row, start_date, end_date):
    latitude = row['Latitude']
    longitude = row['Longitude']
    return allCombined(longitude, latitude, start_date, end_date)


def main():
    file_path = 'ShorterData.csv'  # Replace with your CSV file path
    df = pd.read_csv(file_path)

    # Define start and end dates
    start_date = '2022'
    end_date = '2024'

    # Lists to store results
    results = {
        'Albedo': [],
        'MeanAAI': [],
        'VegetationRatio': [],
        'PopulationDensity': [],
        'ProtectedAreaRatio': [],
        'SlopeTopography': [],
        'CloudCover': [],
        'LandSurfaceTemperature': [],
        'AverageDownwardRadiation': [],
    }

    # Parallel processing with 64 cores
    with ProcessPoolExecutor(max_workers=8) as executor:
        futures = {executor.submit(process_row, row, start_date, end_date): row for index, row in df.iterrows()}

        # Using tqdm to visualize the progress
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
            try:
                alb, aai, veg_ratio, pop_density, proc_area, slope, cloud, land_temp, downward_rad = future.result()

                # Append results to lists
                results['Albedo'].append(alb)
                results['MeanAAI'].append(aai)
                results['VegetationRatio'].append(veg_ratio)
                results['PopulationDensity'].append(pop_density)
                results['ProtectedAreaRatio'].append(proc_area)
                results['SlopeTopography'].append(slope)
                results['CloudCover'].append(cloud)
                results['LandSurfaceTemperature'].append(land_temp)
                results['AverageDownwardRadiation'].append(downward_rad)
            except Exception as e:
                print(f"Error processing row: {e}")

    # Add results back to the DataFrame
    for key in results:
        df[key] = results[key]

    # Save the updated DataFrame back to the CSV file
    output_file_path = 'solarFarmData_processed.csv'
    df.to_csv(output_file_path, index=False)
    print(f"Updated {output_file_path} with calculated values.")


if __name__ == "__main__":
    main()


Processing:   0%|          | 0/9999 [00:00<?, ?it/s]/home/jovyan/.local/lib/python3.10/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for ECMWF/ERA5_LAND/MONTHLY! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_MONTHLY

  warnings.warn(warning, category=DeprecationWarning)
Processing:   0%|          | 1/9999 [00:00<1:46:31,  1.56it/s]/home/jovyan/.local/lib/python3.10/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for ECMWF/ERA5_LAND/MONTHLY! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_MONTHLY

  warnings.warn(warning, category=DeprecationWarning)
Processing:   0%|          | 2/9999 [00:01<1:33:49,  1.78it/s]/home/jovyan/.local/lib/python3.10/site-packages/ee/deprecation.

Updated solarFarmData_processed.csv with calculated values.
